In [2]:
from backend.data_loader import load_data
data = load_data()
print("Events:", len(data["events"]), "Jobs:", len(data["jobs"]))

Events: 130 Jobs: 233


In [6]:
import os
os.chdir(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect") 

In [8]:
from backend.data_loader import load_data
data = load_data()
print("Events:", len(data["events"]), "Jobs:", len(data["jobs"]))


Events: 130 Jobs: 233


In [2]:
# === CELL 1: paths ===
from pathlib import Path

# ⬇️ set your project root ONCE here
PROJECT_ROOT = Path(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect")
DATA_DIR = PROJECT_ROOT / "backend" / "data"

print("Project root:", PROJECT_ROOT)
print("Data dir:", DATA_DIR)
for p in DATA_DIR.glob("*"):
    print(" -", p.name)


Project root: C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect
Data dir: C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect\backend\data
 - berlin_tech_events.csv
 - berlin_tech_jobs.csv
 - german_courses_berlin.csv


In [3]:
# === CELL 2: utilities to read/normalize your CSVs ===
import pandas as pd

def smart_read_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        return pd.DataFrame()
    for enc in ("utf-8", "utf-16", "cp1252", "latin-1"):
        try:
            return pd.read_csv(path, sep=None, engine="python", encoding=enc)
        except Exception:
            pass
    return pd.read_csv(path)  # last resort

def normalize_common(df: pd.DataFrame) -> pd.DataFrame:
    """Make sure every dataset has the same structure and a _blob search column."""
    if df is None or df.empty:
        return pd.DataFrame(columns=["title","org","address","date","link","category","lat","lon","_blob"])

    df = df.copy()

    # handle coordinate columns
    for c in ("lat","lon","latitude","longitude"):
        if c in df.columns:
            try:
                df[c] = pd.to_numeric(df[c], errors="coerce")
            except Exception:
                pass
    if "lat" not in df and "latitude" in df:
        df["lat"] = df["latitude"]
    if "lon" not in df and "longitude" in df:
        df["lon"] = df["longitude"]

    # ensure required text columns exist
    for col in ["title","org","address","date","link","category"]:
        if col not in df.columns:
            df[col] = ""

    # ✅ robust way to find object/string columns
    try:
        dtypes_series = getattr(df, "dtypes", None)
        if dtypes_series is not None:
            obj_cols = [c for c, t in dtypes_series.items() if "object" in str(t)]
        else:
            obj_cols = list(df.columns)
    except Exception:
        obj_cols = list(df.columns)

    # create searchable blob column
    df["_blob"] = df[obj_cols].astype(str).apply(lambda s: " | ".join(s), axis=1).str.lower()

    return df


def load_events_nb(data_dir: Path) -> pd.DataFrame:
    path = data_dir / "berlin_tech_events.csv"
    df = smart_read_csv(path)
    if df.empty: return df
    # your sheet: Title | Link | Date & Time | Location
    mapping = {"Title":"title","Link":"link","Date & Time":"date","Location":"address"}
    # also accept lowercase
    mapping.update({"title":"title","link":"link","date & time":"date","location":"address"})
    df = df.rename(columns=mapping)
    return normalize_common(df)

def load_jobs_nb(data_dir: Path) -> pd.DataFrame:
    path = data_dir / "berlin_tech_jobs.csv"
    df = smart_read_csv(path)
    if df.empty: return df
    # map common columns
    mapping = {}
    for src, dst in [
        ("title","title"),
        ("company","org"),
        ("location","address"),
        ("date_posted","date"),
        ("job_url_direct","link"),
        ("job_url","link"),
        ("job_type","category"),
        ("company_industry","category"),
    ]:
        if src in df.columns: mapping[src] = dst
    df = df.rename(columns=mapping)
    return normalize_common(df)

def pick_col(cols, *candidates):
    lc = [c.lower() for c in cols]
    for want in candidates:
        w = want.lower()
        for i, name in enumerate(lc):
            if w in name:
                return cols[i]
    return None

def load_courses_nb(data_dir: Path) -> pd.DataFrame:
    path = data_dir / "german_courses_berlin.csv"   # your new file
    df = smart_read_csv(path)
    if df.empty: return df
    cols = list(df.columns)
    title   = pick_col(cols, "course title","course","title","name","program")
    org     = pick_col(cols, "provider","institution","institute","school","center","organisation","organization")
    address = pick_col(cols, "address","location","campus","bezirk","district")
    date    = pick_col(cols, "date","start","when","schedule","term","beginn")
    link    = pick_col(cols, "link","url","website","web","page")
    category= pick_col(cols, "category","level","type")
    mapping = {}
    if title:    mapping[title]    = "title"
    if org:      mapping[org]      = "org"
    if address:  mapping[address]  = "address"
    if date:     mapping[date]     = "date"
    if link:     mapping[link]     = "link"
    if category: mapping[category] = "category"
    if "title" not in mapping and cols:   # ultimate fallback
        mapping[cols[0]] = "title"
    df = df.rename(columns=mapping)
    return normalize_common(df)

def load_all_nb(data_dir: Path):
    return {
        "events":  load_events_nb(data_dir),
        "jobs":    load_jobs_nb(data_dir),
        "courses": load_courses_nb(data_dir),
    }


In [4]:
data = load_all_nb(DATA_DIR)
for k in ("events","jobs","courses"):
    print(f"{k.title()}: {len(data[k])} rows")
print("\nCourses columns:", list(data["courses"].columns)[:20])
print(data["courses"][["title","org","address","date","link","category"]].head(5).to_string(index=False))


Events: 130 rows
Jobs: 233 rows
Courses: 97 rows

Courses columns: ['title', 'title', 'category', 'date', 'duration', 'price', 'schedule', 'address', 'link', 'phone', 'requirements', 'completion', 'registration', 'appointment_url', 'booking_url', 'org', '_blob']
                             title                           title org                              address                            date                                                                       link category
     MIQR Berlin - Trachenbergring BAMF Integration Course (A1-B1)     Trachenbergring 93-93a, 12249 Berlin            After placement test https://www.mitteldeutsches-institut.de/en/integration-course-BAMF-Berlin/ A1 to B1
MIQR Berlin - Prenzlauer Promenade BAMF Integration Course (A1-B1)             Prenzlauer Promenade, Berlin            After placement test https://www.mitteldeutsches-institut.de/en/integration-course-BAMF-Berlin/ A1 to B1
                        IIK Berlin BAMF Integration Course (A1-B1

In [11]:
!pip install fastapi uvicorn nest_asyncio


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import nest_asyncio
import uvicorn

# This line allows uvicorn to run inside Jupyter without blocking
nest_asyncio.apply()

# Start your backend app directly
uvicorn.run("backend.main:app", host="127.0.0.1", port=8000, reload=True)


INFO:     Will watch for changes in these directories: ['c:\\Users\\krupa\\Desktop\\Bootcamp\\project_keiz_connect']
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Started reloader process [6556] using StatReload
INFO:     Stopping reloader process [6556]


In [5]:
import nest_asyncio, uvicorn
nest_asyncio.apply()
uvicorn.run("backend.main:app", host="127.0.0.1", port=8000, reload=False, log_level="debug")


ModuleNotFoundError: No module named 'backend'

In [1]:
from pathlib import Path
import os, sys

# 🔴 CHANGE this to your project root folder:
ROOT = Path(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect")

# make it the working directory and ensure it's on sys.path
os.chdir(ROOT)
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("CWD:", Path.cwd())
print("Has backend?", (ROOT / "backend").is_dir())
print("Has backend/main.py?", (ROOT / "backend" / "main.py").is_file())


CWD: C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect
Has backend? True
Has backend/main.py? True


In [2]:
# Create an empty __init__.py if it's missing so Python treats 'backend' as a package
(ROOT / "backend" / "__init__.py").touch()
print("backend/__init__.py present:", (ROOT / "backend" / "__init__.py").is_file())

backend/__init__.py present: True


In [3]:
# --- Step 2 & 3: ensure backend package works and start the FastAPI server ---

import nest_asyncio, uvicorn
from pathlib import Path

# make sure the backend folder is a Python package
Path("backend/__init__.py").touch()

# allow uvicorn to run inside Jupyter
nest_asyncio.apply()

# import your FastAPI app directly
from backend.main import app

# start the API server
uvicorn.run(app, host="127.0.0.1", port=8000, reload=False, log_level="info")


AttributeError: 'DataFrame' object has no attribute 'dtype'

In [4]:
from pathlib import Path
import os, sys

ROOT = Path(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect")
os.chdir(ROOT)
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("CWD:", Path.cwd())
print("Has backend?", (ROOT / "backend").is_dir())
print("Has backend/main.py?", (ROOT / "backend" / "main.py").is_file())


CWD: C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect
Has backend? True
Has backend/main.py? True


In [1]:
from backend.data_loader import load_data
d = load_data()
print("Rows:", len(d["events"]), len(d["jobs"]), len(d["courses"]))
print("Courses columns:", list(d["courses"].columns)[:12])
print(d["courses"][["title","org","address","date","link","category"]].head(5).to_string(index=False))



ModuleNotFoundError: No module named 'backend'

In [2]:
from pathlib import Path
import os, sys, importlib

# 1) Point to your project root (folder that contains /backend and /widget)
ROOT = Path(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect")

# 2) Make it the working directory and add to sys.path
os.chdir(ROOT)
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

# 3) Ensure backend is a package
(Path("backend/__init__.py")).touch()

# 4) Debug printouts
print("CWD:", Path.cwd())
print("Has backend?", (ROOT/"backend").is_dir())
print("Has backend/data_loader.py?", (ROOT/"backend"/"data_loader.py").is_file())

# 5) Clear any old cached imports and try import
importlib.invalidate_caches()


CWD: C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect
Has backend? True
Has backend/data_loader.py? True


In [3]:
from backend.data_loader import load_data
d = load_data()
print("Rows:", len(d["events"]), len(d["jobs"]), len(d["courses"]))
print("Courses columns:", list(d["courses"].columns)[:12])
print(d["courses"][["title","org","address","date","link","category"]].head(5).to_string(index=False))


Rows: 130 233 97
Courses columns: ['title', 'title', 'category', 'date', 'duration', 'price', 'schedule', 'address', 'link', 'phone', 'requirements', 'completion']
                             title                           title org                              address                            date                                                                       link category
     MIQR Berlin - Trachenbergring BAMF Integration Course (A1-B1)     Trachenbergring 93-93a, 12249 Berlin            After placement test https://www.mitteldeutsches-institut.de/en/integration-course-BAMF-Berlin/ A1 to B1
MIQR Berlin - Prenzlauer Promenade BAMF Integration Course (A1-B1)             Prenzlauer Promenade, Berlin            After placement test https://www.mitteldeutsches-institut.de/en/integration-course-BAMF-Berlin/ A1 to B1
                        IIK Berlin BAMF Integration Course (A1-B1)              Berlin (multiple locations)            After placement test                         

In [19]:
from fastapi.middleware.cors import CORSMiddleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True,
    allow_methods=["*"], allow_headers=["*"],
)

RuntimeError: Cannot add middleware after an application has started

In [ ]:
import threading, http.server, socketserver, os
os.chdir(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect")
def run_widget(port=5510):  # use 5510 if 5500 is busy
    with socketserver.TCPServer(("", port), http.server.SimpleHTTPRequestHandler) as httpd:
        print(f"Widget at http://127.0.0.1:{port}/widget/widget.html")
        httpd.serve_forever()
threading.Thread(target=run_widget, daemon=True, kwargs={"port": 5510}).start()

Exception in thread Thread-13 (run_widget):
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\krupa\AppData\Local\Temp\ipykernel_14092\3490107946.py", line 4, in run_widget


  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\socketserver.py", line 457, in __init__
    self.server_bind()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\socketserver.py", line 478, in server_bind
    self.socket.bind(self.server_address)
OSError: [WinError 10048] Only one usage of each socket address (protocol/network address/port) is normally permitted


In [4]:
import importlib; importlib.invalidate_caches()

In [5]:
import nest_asyncio, uvicorn
from backend.main import app  # this will now work
nest_asyncio.apply()
uvicorn.run(app, host="127.0.0.1", port=8000, reload=False, log_level="info")


INFO:     Started server process [14092]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:58732 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:58732 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:51104 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:51104 - "GET /redoc HTTP/1.1" 200 OK
INFO:     127.0.0.1:51104 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:51104 - "GET /chat HTTP/1.1" 405 Method Not Allowed


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [14092]


In [13]:
from pathlib import Path
import os, sys, importlib

ROOT = Path(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect")
os.chdir(ROOT)
if str(ROOT) not in sys.path: sys.path.insert(0, str(ROOT))
(Path("backend/__init__.py")).touch()
importlib.invalidate_caches()

print("CWD:", ROOT)
print("backend/main.py exists?", (ROOT/"backend"/"main.py").is_file())


CWD: C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect
backend/main.py exists? True


In [17]:
# Start FastAPI (8000) + widget (5500) in background threads

import nest_asyncio, uvicorn, threading, socketserver, http.server, time
from backend.main import app

nest_asyncio.apply()

# --- FastAPI in background with a controllable server object
api_config = uvicorn.Config(app, host="127.0.0.1", port=8000, log_level="info")
api_server = uvicorn.Server(api_config)

def _run_api():
    # runs until api_server.should_exit = True
    api_server.run()

api_thread = threading.Thread(target=_run_api, daemon=True)
api_thread.start()

# --- Widget static server in background
class QuietHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, fmt, *args):  # keep notebook output quiet
        pass

widget_port = 5510
widget_httpd = socketserver.TCPServer(("", widget_port), QuietHandler)

def _run_widget():
    os.chdir(ROOT)   # serve from project root so /widget exists
    widget_httpd.serve_forever()

widget_thread = threading.Thread(target=_run_widget, daemon=True)
widget_thread.start()

time.sleep(1)
print(f"✅ API running at   http://127.0.0.1:8000")
print(f"✅ Widget running at http://127.0.0.1:{widget_port}/widget/widget.html")


Task exception was never retrieved
future: <Task finished name='Task-78' coro=<Server.serve() done, defined at C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\asyncio\base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted



✅ API running at   http://127.0.0.1:8000
✅ Widget running at http://127.0.0.1:5510/widget/widget.html


Task exception was never retrieved
future: <Task finished name='Task-81' coro=<Server.serve() done, defined at C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\asyncio\base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted



In [11]:
import requests
print("health:", requests.get("http://127.0.0.1:8000/health").text)
print("openapi:", requests.get("http://127.0.0.1:8000/openapi.json").status_code)

health: {"ok":true,"events":130,"jobs":233,"courses":97}
openapi: 200


In [24]:
# Stop FastAPI
api_server.should_exit = True
if api_thread.is_alive():
    api_thread.join(timeout=3)

# Stop widget server
try:
    widget_httpd.shutdown()
    widget_httpd.server_close()
except Exception:
    pass

print("🛑 Servers stopped.")


🛑 Servers stopped.


In [ ]:
# --- Start/Restart API (8000) + Widget (free port) in background threads ---

from pathlib import Path
import os, sys, threading, http.server, socketserver, socket, time
import nest_asyncio, uvicorn

# Project root
ROOT = Path(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect")
os.chdir(ROOT)
if str(ROOT) not in sys.path: sys.path.insert(0, str(ROOT))

# Try to stop previously running servers (if any)
try:
    api_server.should_exit = True
except Exception:
    pass

try:
    widget_httpd.shutdown()
    widget_httpd.server_close()
except Exception:
    pass

# Import app AFTER sys.path is set and root is CWD
from backend.main import app

# --- API ---
nest_asyncio.apply()
api_config = uvicorn.Config(app, host="127.0.0.1", port=8010, log_level="info")
api_server = uvicorn.Server(api_config)
api_thread = threading.Thread(target=api_server.run, daemon=True)
api_thread.start()

# --- Widget (auto free port) ---
class QuietHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, *args): pass  # keep notebook output clean

os.chdir(ROOT)  # serve from project root
widget_httpd = socketserver.TCPServer(("", 0), QuietHandler)  # 0 = OS picks a free port
widget_port = widget_httpd.server_address[1]
widget_thread = threading.Thread(target=widget_httpd.serve_forever, daemon=True)
widget_thread.start()

time.sleep(1)
print(f"✅ API running at    http://127.0.0.1:8000")
print(f"✅ Widget available: http://127.0.0.1:{widget_port}/widget/widget.html")


INFO:     Started server process [14092]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


✅ API running at    http://127.0.0.1:8000
✅ Widget available: http://127.0.0.1:53732/widget/widget.html


In [26]:
import requests
print(requests.get("http://127.0.0.1:8000/health").json())
print(requests.post("http://127.0.0.1:8000/chat", json={"message":"jobs in berlin"}).json())
print(requests.post("http://127.0.0.1:8000/chat", json={"message":"top 10 german courses list"}).json())

{'ok': True, 'events': 130, 'jobs': 233, 'courses': 97}


Task exception was never retrieved
future: <Task finished name='Task-94' coro=<Server.serve() done, defined at C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py:69> exception=SystemExit(1)>
Traceback (most recent call last):
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py", line 164, in startup
    server = await loop.create_server(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\asyncio\base_events.py", line 1584, in create_server
    raise OSError(err.errno, msg) from None
OSError: [Errno 10048] error while attempting to bind on address ('127.0.0.1', 8000): [winerror 10048] only one usage of each socket address (protocol/network address/port) is normally permitted



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [28]:
# Try to stop any servers we started earlier in this notebook
try: api_server.should_exit = True
except: pass
try:
    widget_httpd.shutdown(); widget_httpd.server_close()
except: pass

In [31]:
from pathlib import Path
import os, sys, nest_asyncio, uvicorn, threading, time

ROOT = Path(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect")
os.chdir(ROOT)
if str(ROOT) not in sys.path: sys.path.insert(0, str(ROOT))

from backend.main import app  # <-- your safe main.py with CORS + handle_chat_safe

nest_asyncio.apply()
API_PORT = 8010  # <— use 8010 to avoid 8000 conflicts
api_config = uvicorn.Config(app, host="127.0.0.1", port=API_PORT, log_level="info")
api_server = uvicorn.Server(api_config)
api_thread = threading.Thread(target=api_server.run, daemon=True)
api_thread.start()

time.sleep(1)
print(f"✅ API running at http://127.0.0.1:{API_PORT}")


INFO:     Started server process [14092]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8010 (Press CTRL+C to quit)


✅ API running at http://127.0.0.1:8010


INFO:     127.0.0.1:54291 - "GET / HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54291 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:54355 - "GET /health HTTP/1.1" 200 OK
INFO:     127.0.0.1:54356 - "POST /chat HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\krupa\AppData\Local\Packages\Python

INFO:     127.0.0.1:54357 - "POST /chat HTTP/1.1" 500 Internal Server Error


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\protocols\http\h11_impl.py", line 403, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\middleware\proxy_headers.py", line 60, in __call__
    return await self.app(scope, receive, send)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\krupa\AppData\Local\Packages\Python

In [32]:
import http.server, socketserver

class QuietHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, *args): pass

os.chdir(ROOT)  # serve from project root so /widget exists
widget_httpd = socketserver.TCPServer(("", 0), QuietHandler)  # 0 = OS picks a free port
WIDGET_PORT = widget_httpd.server_address[1]
widget_thread = threading.Thread(target=widget_httpd.serve_forever, daemon=True)
widget_thread.start()

print(f"✅ Widget at http://127.0.0.1:{WIDGET_PORT}/widget/widget.html")


✅ Widget at http://127.0.0.1:54301/widget/widget.html


In [33]:
import http.server, socketserver

class QuietHandler(http.server.SimpleHTTPRequestHandler):
    def log_message(self, *args): pass

os.chdir(ROOT)  # serve from project root so /widget exists
widget_httpd = socketserver.TCPServer(("", 0), QuietHandler)  # 0 = OS picks a free port
WIDGET_PORT = widget_httpd.server_address[1]
widget_thread = threading.Thread(target=widget_httpd.serve_forever, daemon=True)
widget_thread.start()

print(f"✅ Widget at http://127.0.0.1:{WIDGET_PORT}/widget/widget.html")


✅ Widget at http://127.0.0.1:54341/widget/widget.html


In [34]:
import requests
API = "http://127.0.0.1:8010"
r1 = requests.get(f"{API}/health")
print("health:", r1.status_code, r1.text)

r2 = requests.post(f"{API}/chat", json={"message":"jobs in berlin"})
print("chat jobs:", r2.status_code, r2.text)

r3 = requests.post(f"{API}/chat", json={"message":"top 10 german courses list"})
print("chat courses:", r3.status_code, r3.text)

health: 200 {"ok":true,"events":130,"jobs":233,"courses":97}
chat jobs: 500 Internal Server Error
chat courses: 500 Internal Server Error


In [3]:
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from pathlib import Path
import pandas as pd
import re

# ------------------ PATHS ------------------
ROOT = Path(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect")
DATA_DIR = ROOT / "backend" / "data"

EVENTS_CSV  = DATA_DIR / "berlin_tech_events.csv"      # Title | Link | Date & Time | Location
JOBS_CSV    = DATA_DIR / "berlin_tech_jobs.csv"        # id | site | job_url | job_url_direct | title | company | location | date_posted | ...
COURSES_CSV = DATA_DIR / "german_courses_berlin.csv"   # (your German courses file name; change if needed)

# ------------------ UTILS ------------------
def _read_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        return pd.DataFrame()
    for enc in ("utf-8-sig", "utf-8", None):
        try:
            return pd.read_csv(path, encoding=enc) if enc else pd.read_csv(path)
        except Exception:
            continue
    # very robust last attempt
    return pd.read_csv(path, encoding_errors="ignore")

def _ensure_df(x) -> pd.DataFrame:
    """Make sure we always return a DataFrame (even if x is a Series/list/None)."""
    if isinstance(x, pd.DataFrame):
        df = x.copy()
    elif isinstance(x, pd.Series):
        df = x.to_frame()
    else:
        try:
            df = pd.DataFrame(x)
        except Exception:
            df = pd.DataFrame()
    df.columns = [str(c).strip() for c in df.columns]
    return df

def _coerce_numeric(s):
    """Coerce a 1-D object to numeric series; return a Series (or None on failure)."""
    try:
        return pd.to_numeric(s, errors="coerce")
    except Exception:
        try:
            return pd.Series(pd.to_numeric(list(s), errors="coerce"))
        except Exception:
            return None

def _norm_common(df: pd.DataFrame) -> pd.DataFrame:
    """Light cleanup + create lat/lon if found, without using .dtype anywhere."""
    df = _ensure_df(df)
    if df.empty:
        return df

    # Clean string/object columns
    for c in df.columns:
        try:
            df[c] = df[c].astype(str).str.strip()
        except Exception:
            pass

    # Find any plausible latitude/longitude columns and create lat/lon
    # (won’t crash even if 2-D slices happen)
    lat_candidates = [c for c in df.columns if c.lower() in {"lat","latitude"} or "latitude" in c.lower()]
    lon_candidates = [c for c in df.columns if c.lower() in {"lon","lng","longitude"} or "longitude" in c.lower()]

    lat_col = lat_candidates[0] if lat_candidates else None
    lon_col = lon_candidates[0] if lon_candidates else None

    if lat_col and lon_col:
        s_lat = _coerce_numeric(df[lat_col])
        s_lon = _coerce_numeric(df[lon_col])
        if s_lat is not None and s_lon is not None:
            df["lat"] = s_lat
            df["lon"] = s_lon

    return df

# ------------------ LOADERS ------------------
def load_events() -> pd.DataFrame:
    df = _ensure_df(_read_csv(EVENTS_CSV))
    if df.empty:
        return df
    # Expected: Title | Link | Date & Time | Location
    mapping = {}
    for c in df.columns:
        lc = c.lower()
        if lc.startswith("title"): mapping[c] = "title"
        elif lc.startswith("link"): mapping[c] = "link"
        elif "date" in lc or "time" in lc: mapping[c] = "date"
        elif "loc" in lc or "venue" in lc or "place" in lc: mapping[c] = "address"
        elif "org" in lc: mapping[c] = "org"
    if mapping:
        df = df.rename(columns=mapping)
    df = _norm_common(df)
    if "org" not in df.columns:
        df["org"] = ""
    keep = [c for c in ["title","org","address","date","link","lat","lon"] if c in df.columns]
    return df[keep] if keep else df

def load_jobs() -> pd.DataFrame:
    df = _ensure_df(_read_csv(JOBS_CSV))
    if df.empty:
        return df
    mapping = {}
    for c in df.columns:
        lc = c.lower()
        if lc == "title": mapping[c] = "title"
        elif lc == "company": mapping[c] = "org"
        elif "location" in lc: mapping[c] = "address"
        elif lc in ("job_url","job_url_direct"): mapping[c] = "link"
        elif "date" in lc: mapping[c] = "date"
        elif lc in ("lat","latitude"): mapping[c] = "lat"
        elif lc in ("lon","lng","longitude"): mapping[c] = "lon"
    if mapping:
        df = df.rename(columns=mapping)
    df = _norm_common(df)
    if "org" not in df.columns:
        df["org"] = ""
    keep = [c for c in ["title","org","address","date","link","lat","lon"] if c in df.columns]
    return df[keep] if keep else df

def load_courses() -> pd.DataFrame:
    df = _ensure_df(_read_csv(COURSES_CSV))
    if df.empty:
        return df
    mapping = {}
    for c in df.columns:
        lc = c.lower()
        if "title" in lc or "course" in lc: mapping[c] = "title"
        elif lc in {"org","organisation","organization","provider","school","institute","institute_name"}:
            mapping[c] = "org"
        elif "address" in lc or "location" in lc: mapping[c] = "address"
        elif "date" in lc or "start" in lc: mapping[c] = "date"
        elif "link" in lc or "url" in lc: mapping[c] = "link"
        elif "category" in lc or "level" in lc: mapping[c] = "category"
        elif lc in ("lat","latitude"): mapping[c] = "lat"
        elif lc in ("lon","lng","longitude"): mapping[c] = "lon"
    if mapping:
        df = df.rename(columns=mapping)
    df = _norm_common(df)
    for need in ["org","category"]:
        if need not in df.columns:
            df[need] = ""
    keep = [c for c in ["title","org","address","date","link","category","lat","lon"] if c in df.columns]
    return df[keep] if keep else df

DATA = {
    "events":  load_events(),
    "jobs":    load_jobs(),
    "courses": load_courses(),
}

# ------------------ SIMPLE CHAT ------------------
NEIGHBORHOODS = [
    "mitte","kreuzberg","friedrichshain","neukölln","charlottenburg","wedding",
    "moabit","prenzlauer","spandau","schöneberg","tempelhof","lichtenberg",
    "pankow","treptow","köpenick","kopenick"
]

def choose_table(q: str) -> str:
    ql = (q or "").lower()
    if any(w in ql for w in ["course","courses","sprach","german","class","schule","schule","kurs"]):
        return "courses"
    if any(w in ql for w in ["job","jobs","hiring","role","position","apply","career"]):
        return "jobs"
    return "events"

def extract_area(q: str) -> str | None:
    ql = (q or "").lower()
    m = re.search(r"\b(?:in|near)\s+([a-zäöüß\-]+)", ql)
    cand = m.group(1) if m else ""
    for n in NEIGHBORHOODS:
        if n in ql or n == cand:
            return n
    return None

def build_markers(df: pd.DataFrame, limit=50):
    if df is None or df.empty:
        return []
    out = []
    for _, r in df.head(limit).iterrows():
        lat = r.get("lat"); lon = r.get("lon")
        try:
            latf = float(lat) if pd.notna(lat) else None
            lonf = float(lon) if pd.notna(lon) else None
        except Exception:
            latf = lonf = None
        if latf is None or lonf is None:
            continue
        # stay roughly inside Berlin
        if 52.2 <= latf <= 52.7 and 13.0 <= lonf <= 13.8:
            out.append({
                "lat": latf, "lon": lonf,
                "title": r.get("title",""),
                "org": r.get("org",""),
                "address": r.get("address",""),
                "date": r.get("date",""),
                "link": r.get("link","")
            })
    return out

def handle_chat_safe(message: str) -> dict:
    """Never raises: always returns a friendly answer."""
    try:
        intent = choose_table(message)
        df = DATA.get(intent)
        if df is None or df.empty:
            return {"reply": f"Sorry, I have no {intent} data.", "results": [], "markers": [], "intent": intent}

        area = extract_area(message)
        filt = df
        if area and "address" in df.columns:
            sub = df[df["address"].str.lower().str.contains(area, na=False)]
            if sub.empty and "title" in df.columns:
                sub = df[df["title"].str.lower().str.contains(area, na=False)]
            if not sub.empty:
                filt = sub

        top = _ensure_df(filt).head(10)
        keep_cols = [c for c in ["title","org","address","date","link","category"] if c in top.columns]
        results = _ensure_df(top[keep_cols]).fillna("").to_dict(orient="records") if keep_cols else []
        markers = build_markers(top, limit=10)

        if not results:  # last resort: first 10 of whole table
            top = _ensure_df(df).head(10)
            results = _ensure_df(top[keep_cols]).fillna("").to_dict(orient="records") if keep_cols else []
            markers = build_markers(top, limit=10)

        reply = f"Found {len(results)} {intent}{' in ' + area if area else ''}."
        return {"reply": reply, "results": results, "markers": markers, "intent": intent}
    except Exception as e:
        return {
            "reply": "Sorry, something went wrong. Try: 'events in mitte', 'AI jobs', or 'german courses in neukölln'.",
            "results": [], "markers": [], "intent": "unknown", "error": type(e).__name__
        }

# ------------------ FASTAPI APP ------------------
app = FastAPI(title="Kiez Connect (single file)", docs_url=None, redoc_url="/docs")

# CORS (so the widget on 55xx can call API on 80xx)
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True,
    allow_methods=["*"], allow_headers=["*"],
)

class ChatIn(BaseModel):
    message: str

@app.get("/health")
def health():
    def count(df):
        return 0 if df is None or df.empty else len(df)
    return {"ok": True, "events": count(DATA["events"]), "jobs": count(DATA["jobs"]), "courses": count(DATA["courses"])}

@app.post("/chat")
def chat_api(payload: ChatIn):
    return handle_chat_safe(payload.message)


In [5]:
app = FastAPI(
    title="Kiez Connect (single file)",
    docs_url="/docs",     # Swagger (interactive)
    redoc_url="/redoc"    # ReDoc (read-only)
)

In [9]:
from fastapi.middleware.cors import CORSMiddleware

app.add_middleware(
    CORSMiddleware,
    allow_origins=["http://localhost:5500", "http://127.0.0.1:5500", "http://localhost", "http://127.0.0.1"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
uvicorn your_app_module:app --host 127.0.0.1 --port 8000

In [8]:
import requests

API = "http://127.0.0.1:8030"
print(requests.get(f"{API}/health").json())

r = requests.post(f"{API}/chat", json={"message": "events in berlin"})
print(r.status_code, r.text)

{'ok': True, 'events': 130, 'jobs': 233, 'courses': 97}
200 {"reply":"Found 10 events.","results":[{"title":"I Missed Out on AI and NVIDIA, What’s Next?","org":"","address":"Online","date":"Going fast","link":"https://www.eventbrite.com/e/i-missed-out-on-ai-and-nvidia-whats-next-tickets-1510837408359?aff=ebdssbdestsearch"},{"title":"AI & Teens: The Conversation You Can't Afford to Skip","org":"","address":"Online","date":"Fri, Nov 7, 9:00 AM PST","link":"https://www.eventbrite.com/e/ai-teens-the-conversation-you-cant-afford-to-skip-tickets-1766711966049?aff=ebdssbdestsearch"},{"title":"DM To Dollars 3-Day Challenge","org":"","address":"Online","date":"Tue, Oct 28, 12:00 PM EDT","link":"https://www.eventbrite.com/e/dm-to-dollars-3-day-challenge-tickets-1849452404929?aff=ebdssbdestsearch"},{"title":"Berlin Deep Tech Nexus Meetup #7 - TU Berlin - Science & Startups","org":"","address":"Berlin · TU Berlin, EINS - Innovationsplattform und Coworkingspace","date":"Sales end soon","link":"http

In [10]:
%pip install fastapi uvicorn nest_asyncio pyngrok

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import nest_asyncio
import uvicorn

# Initialize app
app = FastAPI()

# Allow connections from your notebook or local HTML files
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# Example endpoint
@app.get("/")
def read_root():
    return {"message": "Hello from FastAPI in Jupyter!"}

# Patch asyncio loop and run server
nest_asyncio.apply()
uvicorn.run(app, host="127.0.0.1", port=8000)

INFO:     Started server process [30732]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60043 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:60061 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:60061 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:60062 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:60164 - "GET / HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [30732]


In [17]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI(
    title="Kiez Connect",
    docs_url="/docs",      # Swagger UI (interactive)
    redoc_url="/redoc",    # ReDoc (read-only)
    openapi_url="/openapi.json"
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True,
    allow_methods=["*"], allow_headers=["*"],
)

# Setup app
app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all for dev
    allow_methods=["*"],
    allow_headers=["*"],
    allow_credentials=True,
)

class ChatRequest(BaseModel):
    message: str

@app.post("/chat")
async def chat_api(payload: ChatRequest):
    return {"reply": f"Echo: {payload.message}", "results": [], "markers": []}

# Patch asyncio for Jupyter
nest_asyncio.apply()

# Run server with KeyboardInterrupt graceful handling
try:
    uvicorn.run(app, host="127.0.0.1", port=8000)
except KeyboardInterrupt:
    print("Server stopped manually")


INFO:     Started server process [30732]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [30732]


In [18]:
# app.py — single-file backend (EVENTS + JOBS only)

from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from pathlib import Path
import pandas as pd
import hashlib
import re
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI(
    title="Kiez Connect",
    docs_url="/docs",      # Swagger UI (interactive)
    redoc_url="/redoc",    # ReDoc (read-only)
    openapi_url="/openapi.json"
)

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True,
    allow_methods=["*"], allow_headers=["*"],
)

# ---------- paths ----------
ROOT = Path(r"C:\Users\krupa\Desktop\Bootcamp\project_keiz_connect\kiez_connect")
DATA_DIR = ROOT / "backend" / "data"

EVENTS_CSV = DATA_DIR / "berlin_tech_events.csv"  # columns: Title | Link | Date & Time | Location
JOBS_CSV   = DATA_DIR / "berlin_tech_jobs.csv"    # columns: id | site | job_url | job_url_direct | title | company | location | date_posted | job_type | is_remote | company_industry | company_url_direct

# ---------- helpers ----------
BB_MIN_LAT, BB_MAX_LAT = 52.40, 52.62
BB_MIN_LON, BB_MAX_LON = 13.20, 13.60

def _read_csv(path: Path) -> pd.DataFrame:
    if not path.exists():
        return pd.DataFrame()
    for enc in ("utf-8-sig","utf-8",None):
        try:
            return pd.read_csv(path, encoding=enc) if enc else pd.read_csv(path)
        except Exception:
            continue
    return pd.read_csv(path, encoding_errors="ignore")

def _ensure_df(x) -> pd.DataFrame:
    if isinstance(x, pd.DataFrame): df = x.copy()
    elif isinstance(x, pd.Series):  df = x.to_frame()
    else:
        try: df = pd.DataFrame(x)
        except Exception: df = pd.DataFrame()
    df.columns = [str(c).strip() for c in df.columns]
    return df

def _coerce_num(s):
    try:
        return pd.to_numeric(s, errors="coerce")
    except Exception:
        try: return pd.Series(pd.to_numeric(list(s), errors="coerce"))
        except Exception: return None

def _fallback_coords(text: str):
    """Deterministic fallback inside Berlin bbox."""
    if not text: text = "berlin"
    h = hashlib.sha256(text.encode("utf-8")).hexdigest()
    a = int(h[:8], 16) / 0xFFFFFFFF
    b = int(h[8:16], 16) / 0xFFFFFFFF
    lat = BB_MIN_LAT + a * (BB_MAX_LAT - BB_MIN_LAT)
    lon = BB_MIN_LON + b * (BB_MAX_LON - BB_MIN_LON)
    return float(lat), float(lon)

def _norm_common(df: pd.DataFrame) -> pd.DataFrame:
    df = _ensure_df(df)
    if df.empty: return df
    # basic string clean
    for c in df.columns:
        try: df[c] = df[c].astype(str).str.strip()
        except Exception: pass
    # optional lat/lon detection
    lat_cands = [c for c in df.columns if c.lower() in {"lat","latitude"} or "latitude" in c.lower()]
    lon_cands = [c for c in df.columns if c.lower() in {"lon","lng","longitude"} or "longitude" in c.lower()]
    if lat_cands and lon_cands:
        s_lat = _coerce_num(df[lat_cands[0]])
        s_lon = _coerce_num(df[lon_cands[0]])
        if s_lat is not None and s_lon is not None:
            df["lat"] = s_lat; df["lon"] = s_lon
    return df

# ---------- loaders (events + jobs) ----------
def load_events() -> pd.DataFrame:
    df = _read_csv(EVENTS_CSV); df = _ensure_df(df)
    if df.empty: return df
    # map your headers: Title | Link | Date & Time | Location
    rename = {}
    for c in df.columns:
        lc = c.lower()
        if lc.startswith("title"): rename[c] = "title"
        elif lc.startswith("link"): rename[c] = "link"
        elif "date" in lc or "time" in lc: rename[c] = "date"
        elif "loc" in lc or "venue" in lc or "place" in lc: rename[c] = "address"
        elif "org" in lc: rename[c] = "org"
    df = df.rename(columns=rename)
    df = _norm_common(df)
    if "org" not in df.columns: df["org"] = ""
    keep = [c for c in ["title","org","address","date","link","lat","lon"] if c in df.columns]
    return df[keep] if keep else df

def load_jobs() -> pd.DataFrame:
    df = _read_csv(JOBS_CSV); df = _ensure_df(df)
    if df.empty: return df
    # your jobs headers mapping
    rename = {}
    for c in df.columns:
        lc = c.lower()
        if lc == "title": rename[c] = "title"
        elif lc == "company": rename[c] = "org"
        elif "location" in lc: rename[c] = "address"
        elif lc in ("job_url","job_url_direct"): rename[c] = "link"
        elif "date" in lc: rename[c] = "date"
        elif lc in ("lat","latitude"): rename[c] = "lat"
        elif lc in ("lon","lng","longitude"): rename[c] = "lon"
    df = df.rename(columns=rename)
    df = _norm_common(df)
    if "org" not in df.columns: df["org"] = ""
    keep = [c for c in ["title","org","address","date","link","lat","lon"] if c in df.columns]
    return df[keep] if keep else df

DATA = {
    "events": load_events(),
    "jobs":   load_jobs(),
}

# ---------- chat logic ----------
NEIGHBORHOODS = [
    "mitte","kreuzberg","friedrichshain","neukölln","charlottenburg","wedding",
    "moabit","prenzlauer","spandau","schöneberg","tempelhof","lichtenberg",
    "pankow","treptow","köpenick","kopenick"
]

def choose_table(q: str) -> str:
    ql = (q or "").lower()
    if any(w in ql for w in ["job","jobs","hiring","role","position","apply","career"]):
        return "jobs"
    return "events"

def extract_area(q: str) -> str | None:
    ql = (q or "").lower()
    m = re.search(r"\b(?:in|near)\s+([a-zäöüß\-]+)", ql)
    cand = m.group(1) if m else ""
    for n in NEIGHBORHOODS:
        if n in ql or n == cand:
            return n
    return None

def build_markers(df: pd.DataFrame, limit=50):
    if df is None or df.empty: return []
    out = []
    for _, r in df.head(limit).iterrows():
        lat, lon = r.get("lat", None), r.get("lon", None)
        try:
            latf = float(lat) if pd.notna(lat) else None
            lonf = float(lon) if pd.notna(lon) else None
        except Exception:
            latf = lonf = None
        if latf is None or lonf is None:
            # fallback pin so the map always shows Berlin
            latf, lonf = _fallback_coords(str(r.get("address","")) or str(r.get("title","")))
        # constrain to Berlin-ish bounds (avoid map jump)
        if not (52.2 <= latf <= 52.7 and 13.0 <= lonf <= 13.8):
            latf, lonf = _fallback_coords(str(r.get("address","")) or str(r.get("title","")))
        out.append({
            "lat": latf, "lon": lonf,
            "title": r.get("title",""),
            "org": r.get("org",""),
            "address": r.get("address",""),
            "date": r.get("date",""),
            "link": r.get("link","")
        })
    return out

def handle_chat_safe(message: str) -> dict:
    """Simple + safe. Never raises. Works for events & jobs."""
    try:
        intent = choose_table(message)
        df = DATA.get(intent)
        if df is None or df.empty:
            return {"reply": f"Sorry, no {intent} data.", "results": [], "markers": [], "intent": intent}

        area = extract_area(message)
        filt = df
        if area and "address" in df.columns:
            sub = df[df["address"].str.lower().str.contains(area, na=False)]
            if sub.empty and "title" in df.columns:
                sub = df[df["title"].str.lower().str.contains(area, na=False)]
            if not sub.empty:
                filt = sub

        top = _ensure_df(filt).head(10)
        keep = [c for c in ["title","org","address","date","link"] if c in top.columns]
        results = _ensure_df(top[keep]).fillna("").to_dict(orient="records") if keep else []
        markers = build_markers(top, limit=10)
        return {
            "reply": f"Found {len(results)} {intent}{' in ' + area if area else ''}.",
            "results": results, "markers": markers, "intent": intent
        }
    except Exception as e:
        return {
            "reply": "Sorry, something went wrong. Try: 'events in mitte' or 'AI jobs'.",
            "results": [], "markers": [], "intent": "unknown", "error": type(e).__name__
        }

# ---------- fastapi ----------
app = FastAPI(title="Kiez Connect — events+jobs", docs_url=None, redoc_url="/docs")
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"], allow_credentials=True,
    allow_methods=["*"], allow_headers=["*"],
)

class ChatIn(BaseModel):
    message: str

@app.get("/health")
def health():
    def count(df):
        return 0 if df is None or df.empty else len(df)
    return {"ok": True, "events": count(DATA["events"]), "jobs": count(DATA["jobs"])}

@app.post("/chat")
def chat_api(payload: ChatIn):
    return handle_chat_safe(payload.message)


Task exception was never retrieved
future: <Task finished name='Task-21' coro=<Server.serve() done, defined at C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py:69> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\main.py", line 580, in run
    server.run()
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\uvicorn\server.py", line 67, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\krupa\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\nest_asyncio.py", line 30, in

In [16]:
import requests
API = "http://127.0.0.1:8031"

print("health:", requests.get(f"{API}/health").json())
r = requests.post(f"{API}/chat", json={"message":"events in mitte"})
print("status:", r.status_code)
print("body:", r.text)

health: {'ok': True, 'events': 130, 'jobs': 233}
status: 200
body: {"reply":"Found 4 events in mitte.","results":[{"title":"Geopolitics in 40 Minutes: A Practical Guide to What’s Changing","org":"","address":"Berlin · The Castle Berlin Mitte","date":"Tue, Nov 4, 6:30 PM","link":"https://www.eventbrite.de/e/geopolitics-in-40-minutes-a-practical-guide-to-whats-changing-tickets-1674678411209?aff=ebdssbdestsearch"},{"title":"KI-EscapeRoom - Spielerisch künstliche Intelligenz für Unternehmen erleben","org":"","address":"Werder (Havel) · Mittelstand-Digital Zentrum Berlin (Standort Werder (Havel))","date":"Tomorrow at 11:30 AM + 81 more","link":"https://www.eventbrite.de/e/ki-escaperoom-spielerisch-kunstliche-intelligenz-fur-unternehmen-erleben-tickets-1521501655409?aff=ebdssbdestsearch"},{"title":"Data Vault 2.1 Boot Camp and Certification – Berlin  (November 2025)(Ger)","org":"","address":"Berlin · Hotel NH Collection Berlin Mitte am Checkpoint Charlie","date":"Mon, Nov 17, 9:00 AM","link"